In [1]:
!nvidia-smi

Thu May 14 13:16:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import pandas as pd
import numpy as np

In [3]:
!pip install pytorch-pretrained-bert

In [4]:
!pip install seqeval

In [0]:
dframe = pd.read_csv("./ner_dataset.csv", encoding="latin1")

In [6]:
dframe.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [0]:
data = dframe.fillna(method="ffill")

In [8]:
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [0]:
class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        self.grouped = self.data.groupby("Sentence #").apply(lambda s:[(w,p,t) for w,p,t in zip(s["Word"].values.tolist(),
                                                                                                s["POS"].values.tolist(),
                                                                                                s["Tag"].values.tolist())])
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
          s = self.grouped["Sentence: {}".format(self.n_sent)]
          self.n_sent += 1
          return s
        except:
          return None

In [10]:
getter = SentenceGetter(data)
getter.get_next()

[('Thousands', 'NNS', 'O'),
 ('of', 'IN', 'O'),
 ('demonstrators', 'NNS', 'O'),
 ('have', 'VBP', 'O'),
 ('marched', 'VBN', 'O'),
 ('through', 'IN', 'O'),
 ('London', 'NNP', 'B-geo'),
 ('to', 'TO', 'O'),
 ('protest', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('war', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('Iraq', 'NNP', 'B-geo'),
 ('and', 'CC', 'O'),
 ('demand', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('withdrawal', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('British', 'JJ', 'B-gpe'),
 ('troops', 'NNS', 'O'),
 ('from', 'IN', 'O'),
 ('that', 'DT', 'O'),
 ('country', 'NN', 'O'),
 ('.', '.', 'O')]

In [11]:
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [0]:
labels = [[s[2] for s in sent] for sent in getter.sentences]

In [13]:
print(np.array(labels))

[list(['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O'])
 list(['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-tim', 'O', 'O', 'O', 'B-org', 'O', 'O', 'O', 'O', 'O'])
 list(['O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'I-geo', 'O'])
 ...
 list(['O', 'B-geo', 'O', 'O', 'B-tim', 'I-tim', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])
 list(['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])
 list(['O', 'B-org', 'I-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-tim', 'I-tim', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])]


In [14]:
tags_vals = list(set(data["Tag"].values))
tag2idx = {t:i for i, t in enumerate(tags_vals)}
tag2idx

{'B-art': 14,
 'B-eve': 10,
 'B-geo': 4,
 'B-gpe': 8,
 'B-nat': 9,
 'B-org': 5,
 'B-per': 1,
 'B-tim': 13,
 'I-art': 11,
 'I-eve': 15,
 'I-geo': 12,
 'I-gpe': 3,
 'I-nat': 7,
 'I-org': 2,
 'I-per': 0,
 'I-tim': 6,
 'O': 16}

In [15]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertModel
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from tqdm import tqdm, trange

Using TensorFlow backend.


In [0]:
MAX_LEN = 75
bs = 32

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [31]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case = True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_texts[0])

['thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'london', 'to', 'protest', 'the', 'war', 'in', 'iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'british', 'troops', 'from', 'that', 'country', '.']


In [0]:
input_ids = \
pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],maxlen=MAX_LEN,dtype="long",truncating="post",padding="post")

attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

tags = \
pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],maxlen=MAX_LEN,value=tag2idx["O"],padding="post",truncating="post",dtype="long")

In [0]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids,tags,random_state=2020,test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=2020,test_size=0.1)

In [0]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [0]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [36]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(tag2idx))
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
      

In [0]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias','gamma','beta']
    optimizer_grouped_parameters = [
      {"params":[p for n,p in param_optimizer if not any(nd in n for nd in no_decay)],
       "weight_decay_rate":0.01},
       {"params":[p for n,p in param_optimizer if any(nd in n for nd in no_decay)],
        "weight_decay_rate":0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params":[p for n,p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [0]:
from seqeval.metrics import f1_score

In [0]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [40]:
epochs = 1
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in tqdm(enumerate(train_dataloader),total=len(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)    #把数据放上GPU
        b_input_ids, b_input_mask, b_labels = batch    #从batch中分别拿出
        loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)  #将数据放入模型中算loss
        print(loss.item())
        loss.backward()         #反向传播计算梯度
        tr_loss += loss.item()  #计算总梯度
        nb_tr_examples += b_input_ids.size(0)  #计算训练样本数
        nb_tr_steps += 1     #计算迭代次数
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)  #梯度裁剪
        optimizer.step()    #更新参数
        model.zero_grad()    #梯度置0
    print("Train loss: {}".format(tr_loss/nb_tr_steps))    #计算平均loss

    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    predictions, true_labels = [], []
    for batch in tqdm(valid_dataloader,total=len(valid_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()    #截断反向传播的梯度流
        label_ids = b_labels.to("cpu").numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])   #标签索引
        true_labels.append(label_ids)

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss / nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]   #p_i为索引
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))



Epoch:   0%|          | 0/1 [00:00<?, ?it/s]


  0%|          | 0/1349 [00:00<?, ?it/s]

2.606849431991577





  0%|          | 1/1349 [00:00<21:51,  1.03it/s]

2.0734140872955322





  0%|          | 2/1349 [00:01<21:16,  1.06it/s]

...

Train loss: 0.27682419358203814





  1%|          | 1/150 [00:00<01:26,  1.73it/s]


  1%|▏         | 2/150 [00:01<01:26,  1.72it/s]


  2%|▏         | 3/150 [00:01<01:26,  1.71it/s]


  3%|▎         | 4/150 [00:02<01:25,  1.70it/s]


  3%|▎         | 5/150 [00:02<01:25,  1.70it/s]


  4%|▍         | 6/150 [00:03<01:24,  1.69it/s]


  5%|▍         | 7/150 [00:04<01:24,  1.69it/s]


  5%|▌         | 8/150 [00:04<01:23,  1.69it/s]


  6%|▌         | 9/150 [00:05<01:23,  1.69it/s]


  7%|▋         | 10/150 [00:05<01:22,  1.69it/s]


  7%|▋         | 11/150 [00:06<01:22,  1.69it/s]


  8%|▊         | 12/150 [00:07<01:21,  1.69it/s]


  9%|▊         | 13/150 [00:07<01:20,  1.69it/s]


  9%|▉         | 14/150 [00:08<01:20,  1.69it/s]


 10%|█         | 15/150 [00:08<01:19,  1.69it/s]


 11%|█         | 16/150 [00:09<01:19,  1.69it/s]


 11%|█▏        | 17/150 [00:10<01:18,  1.69it/s]


 12%|█▏        | 18/150 [00:10<01:17,  1.69it/s]


 13%|█▎        | 19/150 [00:11<01:17,  1.69it/s]


 13%|█▎        | 20/150 [00:11<01:16,

Validation loss: 0.1687830942372481
Validation Accuracy: 0.9148329365079364




Epoch: 100%|██████████| 1/1 [21:03<00:00, 1263.21s/it]

F1-Score: 0.36019667525169746


In [42]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy= 0, 0
nb_eval_step, nb_eval_examples = 0, 0
for batch in tqdm(valid_dataloader, total=len(valid_dataloader)):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    
    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask,labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to("cpu").numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
vaild_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l]

print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))



  0%|          | 0/150 [00:00<?, ?it/s]

  1%|          | 1/150 [00:00<01:41,  1.46it/s]

  1%|▏         | 2/150 [00:01<01:38,  1.50it/s]

  2%|▏         | 3/150 [00:01<01:35,  1.53it/s]

  3%|▎         | 4/150 [00:02<01:33,  1.57it/s]

  3%|▎         | 5/150 [00:03<01:30,  1.59it/s]

  4%|▍         | 6/150 [00:03<01:28,  1.62it/s]

  5%|▍         | 7/150 [00:04<01:27,  1.64it/s]

  5%|▌         | 8/150 [00:04<01:25,  1.66it/s]

  6%|▌         | 9/150 [00:05<01:24,  1.66it/s]

  7%|▋         | 10/150 [00:06<01:23,  1.67it/s]

  7%|▋         | 11/150 [00:06<01:22,  1.68it/s]

  8%|▊         | 12/150 [00:07<01:22,  1.68it/s]

  9%|▊         | 13/150 [00:07<01:21,  1.68it/s]

  9%|▉         | 14/150 [00:08<01:20,  1.69it/s]

 10%|█         | 15/150 [00:09<01:19,  1.69it/s]

 11%|█         | 16/150 [00:09<01:19,  1.69it/s]

 11%|█▏        | 17/150 [00:10<01:18,  1.69it/s]

 12%|█▏        | 18/150 [00:10<01:17,  1.69it/s]

 13%|█▎        | 19/150 [00:11<01:17,  1.69it/s]

 13%|█▎        |

Validation loss: 0.08439154711862405
Validation Accuracy: 0.4574164682539682
Validation F1-Score: 0.005759775228283775
